In [1]:
import os
from ase.io import read, write
from ase import Atoms
import numpy as np
import pandas as pd
from pyExecVaspSlurm import *

/Users/dominicwelti/opt/anaconda3/lib/python3.9/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [40]:
# load configuration with ASE
graphite_base = read('/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Graphite/graphite_AB-stacking.cif', index=':')

# save configuration as POSCAR
path_poscar='/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Graphite/graphite_POSCAR'
write(path_poscar, graphite_base, format='vasp')

In [3]:
# Initialize Job
job = Job('Graphite_D3',poscar_path_local=path_poscar)

job.incar_load('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_reference_files/incar_reference')
job.kpoints_load('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_reference_files/kpoints_reference')

Graphite_D3 Job instance started


In [4]:
job.kpoints_modify('subdivisions along recipr. l.', '8 8 8')

In [2]:
# Initialize Server
server = Server('st_st179390','justus2.uni-ulm.de', '/lustre/home/st/st_us-031400/st_st179390/', '/home/st/st_us-031400/st_st179390/batch_scripts/vasp.6.4.1.sbatch') # all refer to server
sftp, ts = server.connect()

Your OTP: Password: <paramiko.Transport at 0xf5ecca0 (cipher aes128-ctr, 128 bits) (active; 0 open channel(s))>


In [7]:
# submit job for relexation using reference XC (D3-BJ)
server.create_folder(sftp, job.name)
server.potcar_gather(job, sftp, '/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
server.poscar_upload(job, sftp)
server.kpoints_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
server.name_batch(job, ts) # rename job in sbatch file
server.batch_submit(job, ts)

Created POTCAR for C
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Updating name - Exit status of execution command: 0
Submitting batch - Exit status of execution command: 0


In [55]:
# Load result from relaxed DFT-D3BJ and submit jobs for PBE, PBE-B3BJ and LDA asat various c parameters
graphite_base = read('/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Graphite/Relaxed_D3-BJ_POSCAR.txt')

from itertools import product
c_range = np.linspace(0.75,1.5,16)
settings = pd.DataFrame(list(product(['PBE-D3BF','PBE', 'LDA'], c_range)), columns=['XC', 'c-Factor'])
settings['IVDW'], settings['GGA']= '0', 'PE'
settings.loc[settings['XC']=='PBE-D3BF',['IVDW']]='12'
settings.loc[settings['XC']=='LDA',['GGA']]='CA'
cell_base=graphite_base.get_cell()

for i, row in settings.iterrows():   
    # set scaled cell
    graphite_base.set_cell([
            cell_base[0],
            cell_base[1],
            cell_base[2]*row['c-Factor']
        ],
        scale_atoms=True
    )
   
    path_poscar='/Users/dominicwelti/Documents/Master_Thesis_Data_Set/Graphite/graphite_POSCAR_new'
    write(path_poscar, graphite_base, format='vasp')

    # Initialize Job
    job = Job(f"Graphite_DFT/{row['XC']}_{row['c-Factor']}", poscar_path_local=path_poscar)

    job.incar_load('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_reference_files/incar_reference')
    job.kpoints_load('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_reference_files/kpoints_reference')

    job.kpoints_modify('subdivisions along recipr. l.', '8 8 8')
    job.incar_modify('IVDW',row['IVDW'])
    job.incar_modify('GGA',row['GGA'])
    job.incar_modify('ISIF','2')
    
    server.create_folder(sftp, job.name)
    if 'PBE' in row['XC']:
        server.potcar_gather(job, sftp, '/home/st/st_us-031400/st_st179390/mypps/potpaw_PBE')
    elif 'LDA' in row['XC']:
        server.potcar_gather(job, sftp, '/home/st/st_us-031400/st_st179390/mypps/potpaw')
    server.incar_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.poscar_upload(job, sftp)
    server.kpoints_upload(job, sftp, '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary')
    server.name_batch(job, ts) # rename job in sbatch file
    server.batch_submit(job, ts)




Graphite_DFT/PBE-D3BF_0.75 Job instance started
Created POTCAR for C
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Updating name - Exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
Graphite_DFT/PBE-D3BF_0.8 Job instance started
Created POTCAR for C
POTCAR files successfully merged and copied to job folder
INCAR file temporarily saved
INCAR file uploaded
Temporarirly saved INCAR removed
POSCAR file uploaded
KPOINTS file temporarily saved
KPOINTS file uploaded
Temporarirly saved KPOINTS removed
Updating name - Exit status of execution command: 0
Submitting batch - Exit status of execution command: 0
Graphite_DFT/PBE-D3BF_0.85 Job instance started
Created POTCAR for C
POTCAR files successfully merged and copied to job folder
INCAR file temporarily s

In [3]:
# Terminate connection
if sftp: sftp.close()
if ts.active: ts.close()